# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from SAC_Agent import *
from ENV_DETAILS import *
import os
#os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64"

from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-05-08 08:59:15.106472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 08:59:15.186401: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 08:59:15.215828: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 08:59:15.696415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPU
Selected port: 54219


2024-05-08 08:59:17.562069: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 08:59:17.707505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 08:59:17.742757: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 08:59:18.130064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9298473450749419981
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 21874802688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11434643560400663326
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-05-08 08:59:22.049974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-08 08:59:22.050153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-08 08:59:22.050250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-08 08:59:22.050395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-08 08:59:22.050495: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [3]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\BipedalWalker\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/BipedalWalker/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_sac/' # Linux

ENV = "Ant-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [4]:

env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<AntEnv<Ant-v2>>>>

In [5]:

max_steps = env._max_episode_steps
max_steps

1000

In [6]:
s = env.observation_space.sample()
s

array([ 1.01183277, -0.84935299,  0.17322328,  1.36387071, -2.21440172,
        0.23142667,  0.22460939, -1.76664972,  0.32785036,  0.54733435,
       -1.6457494 , -2.53769359,  0.14480469, -1.40492632,  0.47469273,
        0.11935798,  0.67997021,  1.29335923, -0.52284461, -0.06411533,
       -1.83497624, -0.57746062,  0.04302837, -0.65248822,  1.38106218,
        0.41944777, -1.56732062, -0.19780135, -0.63891108, -0.55325713,
        0.52308925,  0.09038974, -0.77988933, -1.15240668, -0.28785562,
       -0.06614999,  0.20195633,  1.09053719,  0.57199304, -0.67643305,
        1.2760276 ,  0.20713408, -0.417529  ,  0.21499038,  0.22786539,
       -0.68340365, -1.02931388,  0.74939501,  0.19893234, -0.42292584,
        0.16049534, -1.42284092,  0.15372774,  0.15539783, -1.47741845,
       -0.64181205, -0.59844535, -1.59828583, -0.19730624,  0.50949868,
       -0.25339652, -0.07126207,  0.03455291,  2.58881126, -0.88802036,
       -0.40773808,  1.62464458,  1.0820146 ,  0.65765172,  0.76

In [7]:
env.observation_space

Box(-inf, inf, (111,), float64)

In [8]:
env.observation_space.shape

(111,)

In [9]:
s = env.reset()
s[0]

0.6993234291330553

In [10]:
a = env.action_space.sample()
a, a.shape, a.reshape((1, len(a))).shape

(array([ 0.04596584,  0.10526047,  0.9153065 , -0.9851934 , -0.05714063,
         0.7082809 ,  0.9613625 ,  0.30562714], dtype=float32),
 (8,),
 (1, 8))

In [11]:
env.step(a)

(array([ 7.14907919e-01,  9.92275071e-01,  7.29042082e-02,  7.90443754e-02,
        -6.18639404e-02,  1.47326621e-01,  3.52684326e-01, -1.12545121e-01,
        -3.87291763e-01,  2.66432539e-01, -4.12086189e-01, -8.11570194e-03,
         4.14331410e-01,  8.22603467e-02, -2.14929258e-01, -5.86654329e-03,
        -3.21103398e-01, -5.66047218e-01, -1.99667376e+00,  6.62403145e+00,
         7.17804625e+00, -4.43479078e-01, -6.24939860e+00,  7.07250183e+00,
        -7.20654656e+00,  4.14938448e-01,  1.15117489e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.0

In [12]:
env.action_space.sample()

array([-0.6013701 , -0.38249943, -0.2612928 ,  0.57558364,  0.14169   ,
        0.34746322,  0.2178693 ,  0.723077  ], dtype=float32)

### Training/Hyperparam run

In [13]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "logs_hyper/fit_sac/"

In [14]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_sac"
    project_name = "keras_tunning_soft"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/sac/",
                        end_of_episode = EPISODES, evaluation_epoch = 4000, training_steps = 1000000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,        
                discount = 0.99,
                #discount_min = 0.95, discount_max = 0.99,      
                      
                #min_lr_actor_critic = 0.00001, max_lr_actor_critic= 0.001,
                lr_actor_min = 0.0001, lr_actor_max = 0.001,
                lr_critic_1_min = 0.0001, lr_critic_1_max = 0.001, 
                
                #dense_unit=256,
                dense_min = 128, dense_max = 400,
                
                environment_name=ENV,
                #tau = 0.005,
                tau_min = 0.001, tau_max = 0.1,
                
                num_layers_act = 2, num_layers_crit =2,
                #max_num_layers_act = 2, max_num_layers_crit=2
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 30,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name,
            #seed=0
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")

        
        run_training(
            training_steps = 1500000,  
            train_epochs = 50,
            evaluation_epoch = 4000,
            environment_name = ENV,
            end_of_episode= EPISODES,
            sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH,
            sucess_criteria_value= SUCESS_CRITERIA_VALUE,
            discount= 0.99, 
            dense_units_act = [256,256],  
            dense_units_crit = [256,256], 
            num_layer_a= 2, 
            num_layer_c= 2, 
            writer = writer, 
            reward_scaler = 1, 
            lr_actor= 0.001,
            lr_critic_1= 0.001, 
            lr_alpha = 0.001, 
            tau = 0.005,
            return_agent = False
            )
        

Reloading Tuner from Hyperparam_kt_sac/keras_tunning_soft/tuner0.json

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
0.00087595        |0.00029228        |lr_actor
0.00067272        |0.00050344        |lr_critic_1
0.00046265        |0.00046415        |lr_alpha
0.029099          |0.045398          |tau
287               |139               |dense_units_act_0
164               |336               |dense_units_act_1
254               |387               |dense_units_crit_0
344               |256               |dense_units_crit_1

Trial number :  2


2024-05-08 08:59:25.192019: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch: 4000 : Reward eval/Train: -2983.706137402726/-20.641672813722405 
Epoch: 8000 : Reward eval/Train: -2991.6895426799592/-34.74351005401232 
Epoch: 12000 : Reward eval/Train: -2992.2227530705677/-112.5667928137562 
Epoch: 16000 : Reward eval/Train: -2285.5363528103303/-244.1993849467787 
Epoch: 20000 : Reward eval/Train: -1836.6106896012138/-359.991413123291 
Epoch: 24000 : Reward eval/Train: -1948.2230742840782/-419.0487067272886 
Epoch: 28000 : Reward eval/Train: -2005.661838837008/-472.51349705114427 
Epoch: 32000 : Reward eval/Train: -1887.3539732126596/-519.1607772206557 
Epoch: 36000 : Reward eval/Train: -1680.409608345798/-541.3992716713119 
Epoch: 40000 : Reward eval/Train: -1516.1032470299929/-550.8048475279087 
Epoch: 44000 : Reward eval/Train: -1381.94935806246/-514.5781323485677 
Epoch: 48000 : Reward eval/Train: -1275.3478248838974/-447.7995644975602 
Epoch: 52000 : Reward eval/Train: -1179.0818328660102/-447.0483133979889 
Epoch: 56000 : Reward eval/Train: -1100.3654

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=5):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:

val_env = gym.make(ENV)
dir = r"Hyperparam_kt_sac"

for trials in tuner.oracle.get_best_trials(num_trials=4):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
    
#env_model = tuner.get_best_models()[0]
best_hps = tuner.get_best_hyperparameters(4)[-1]  # Access best hyperparameter configuration
print(f"Best Hyperparameters: {best_hps}")

env_model = tuner.hypermodel.build(best_hps)  # Build the model with best hyperparameters
env_model = rerun_training(training_steps  = 2000000, 
                           save_factor = 50000,  
                           model_path = './checkpoints/SACagent',
                           sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, 
                           sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                           return_agent = True, 
                           model = env_model
                           )

final_rewards = final_evaluation(env_model, val_env, n_tries=200, exploration=exploration_tech, sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH,  video_name = "./sac_"+exploration_tech+"_video.mp4")
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
training_steps = 2000000
discount  = best_hp["discount"]
lr_actor  = best_hp["lr_actor"]
lr_critic_1  = best_hp["lr_critic_1"]
tau  = best_hp["tau"]

end_of_episode = EPISODES
sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
sucess_criteria_value = SUCESS_CRITERIA_VALUE
save_factor=50000
return_agent = True
reward_scaler = 100
environment_name=ENV

n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
dense_layers_actor = []
for i in range(n_dense_layers_actor):
    dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_'+str(i)])

n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
dense_layers_critic = []
for i in range(n_dense_layers_critic):
    dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

env_model = run_training(training_steps = training_steps,
                         discount = discount,  
                         dense_units_act = dense_layers_actor,  
                         dense_units_crit = dense_layers_critic, 
                         num_layer_a = n_dense_layers_actor, 
                         num_layer_c = n_dense_layers_critic, 
                         writer = writer, 
                         end_of_episode = end_of_episode, 
                         save_factor = save_factor, 
                         sucess_criteria_epochs = sucess_criteria_epochs , 
                         sucess_criteria_value = sucess_criteria_value, 
                         environment_name = environment_name, 
                         reward_scaler = reward_scaler , 
                         return_agent = return_agent,
                         lr_actor = lr_actor, 
                         lr_critic_1 = lr_critic_1, 
                         tau = tau )

In [ ]:
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./sac_"+exploration_tech+"_video.mp4")
